## Notebook: Running UTOPIA Model Step by Step

### Import the necessary libraries and the UTOPIA class.

In [4]:
import os
import json
from utopia.src.utopia import utopia  

ModuleNotFoundError: No module named 'numpy'

#### Step 1: Load Configuration and Data Files

In [ ]:

# Let's load the default configuration and data from the 'data' folder.
config_data = UTOPIA.load_json_file('default_config.json')
data_data = UTOPIA.load_json_file('default_data.json')


In [ ]:

# Display the loaded config and data to ensure they were loaded correctly.
print("Loaded Configuration Data:", config_data)
print("Loaded Input Data:", data_data)



#### Step 2: Initialize the UTOPIA Model with the Loaded Data

In [ ]:

# We now initialize the model by passing the loaded data into the UTOPIA class.
model = UTOPIA(config=config_data, data=data_data)


In [ ]:
# Display the initialized model's key properties to confirm it's set up correctly.
print("Model Initialized:")
print(f"MP Density (kg/m3): {model.MPdensity_kg_m3}")
print(f"Box Name: {model.boxName}")
print(f"Compartment Names: {model.compartment_names}")

#### Step 3: Running the Model



# Here, we can now move to running the model's core logic.
# For this, let's assume there is a method 'run_model' or similar in your class.
# You would invoke it here, like so (modify according to your actual method name):

# model.run_model()  # Uncomment and adapt based on actual method name

# Step 4: Output and Results

# After running the model, you might want to display the results.
# For example, assuming the model returns results as a dictionary or DataFrame:
# results = model.get_results()  # Replace with the actual result retrieval method

# Let's print the results (this part depends on how your model outputs results).
# print("Model Results:", results)

# Step 5: Visualize the Results (Optional)

# If your model has data for visualization, we can plot the results using libraries like matplotlib.
# Example:
# import matplotlib.pyplot as plt

# plt.plot(results['some_data_column'])  # Replace with actual results data
# plt.title("Model Results Over Time")
# plt.xlabel("Time")
# plt.ylabel("Value")
# plt.show()

# Step 6: Advanced Configuration (Optional)

# Allow users to adjust some configuration parameters if needed. This could be useful if you want to modify some settings during runtime.
# For example:
# user_config = {"MPdensity_kg_m3": 1200, "boxName": "New_Box"}  # Modify based on user input
# model.update_config(user_config)  # Assuming a method for updating configuration

# Step 7: Saving the Model Output (Optional)

# If you want users to save the results, add a section for that.
# Example:
# output_file = "model_output.json"
# with open(output_file, "w") as file:
#     json.dump(results, file)
# print(f"Results saved to {output_file}")
